# Assessment: Unit 4
--------------------

Complete the problems below in your copy of the Jupyter Notebook.

## Problem 4.1.

Polymer films are commonly tested by clamping the film between two points, starting at a known separation distance, $l_0$. One of the clamps is then mechanically driven away from the other at a fixed rate. The time, displacement and force are recorded with a microcontroller.

Given a starting clamp separation, $l_0$, and clamp displacement, $\Delta l$, the elongation of the film is given by

$$\text{elongation}=\frac{\Delta l}{l_0}$$

In addition to the maximum force and elongation to break the film, a "yield" point may be observed, where the slope of the force v. elongation curve goes to zero and decreases. This is the result of a "neck" forming in the sample. 

1. Load the data from the `tensile_curve.csv` file into a `pandas.DataFrame`
2. Compute the elongation from the displacement data, assuming a starting separation of 50 cm
3. Smooth the noisy force data by computing the rolling mean over an interval $n=51$, centering the value in the interval
4. Identify the yield point
   - Compute the numerical derivative of the force/elongation curve
   - Find the first zero of the derivative
5. Plot the results of the analysis from 0 to 200% elongation
   - Scatter plot every 500th point of the force, elongation data
   - The smoothed curve as a line
6. Label the yield point on the plot

In [ ]:
# problem 4.1. solution
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')

# read the data
tensile_df = pd.read_csv('../../data/tensile_curve.csv')
tensile_df.head()

In [ ]:
# compute the percent elongation
tensile_df['Elongation'] = 100*(tensile_df['Displacement (mm)'] / 50)

# apply rolling mean
n = 51
tensile_df['Smoothed'] = tensile_df['Force (N)'].rolling(n, center=True).mean()

# compute the derivative on the smoothed data
tensile_df['Derivative'] = tensile_df['Smoothed'].diff() / tensile_df['Elongation'].diff()

# the derivative starts positive, so find the first index where it is below 0
yield_index = tensile_df[tensile_df['Derivative']<0].index.min()

# get the x, y points for the yield point
yield_x = tensile_df.iloc[yield_index]['Elongation']
yield_y = tensile_df.iloc[yield_index]['Force (N)']

# plot the results
fig, ax = plt.subplots()

# use iloc slicing to select every 500th point
ax.scatter(tensile_df['Elongation'].iloc[::500], tensile_df['Force (N)'].iloc[::500], marker='x')

# plot the smoothed curve as a line
ax.plot(tensile_df['Elongation'], tensile_df['Smoothed'], c='black')

ax.set_xlabel('Elongation (%)')
ax.set_ylabel('Force (N)')

# label the yield
ax.annotate('Yield', (yield_x, yield_y), (0.8*yield_x, 1.5*yield_y), arrowprops={'width': 1}, ha='center')

## Problem 4.2.

The `physical_properties` worksheet in `film_testing.xlsx` contains a long-form data set with a variety of measured film properties. Plot the mean "Tensile Modulus" (in MPa, as provided) for each polymer film, using the `ggplot` style.

1. Load the `film_testing.xlsx` file
2. Select the "Tensile Modulus" data
3. Create a pivot table of this measurement, by film id
4. Create a bar chart with the mean values of the modulus, by grade
5. Label the plot axes

In [ ]:
# problem 4.2. solution

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')

# load data and filter
film_df = pd.read_excel('../../data/film_testing.xlsx', sheet_name='physical_properties')
film_df.head()

In [ ]:
modulus_df = film_df[film_df['Property']=='Tensile Modulus']

# pivot to average by film id
modulus_pt = modulus_df.pivot_table(values='Measurement', index='FilmID', aggfunc='mean')

# create a range for the x-values in the plot
x = np.arange(len(modulus_pt))

# plot the bar
fig, ax = plt.subplots()
ax.bar(x, modulus_pt['Measurement'])

# set the x tick marks and labels
ax.set_xticks(x)
ax.set_xticklabels(modulus_pt.index)

# label the plot axes
ax.set_xlabel('Film')
ax.set_ylabel('Tensile Modulus (MPa)')

## Problem 4.3.

A data set containing the results of a heat sealing test was introduced previously in Problem 3.3. Refer back to that problem set for further background. Load the file, `seal_curve.csv`, into a `DataFrame` object. At each test temperature there are 3 replicate measurements. 

1. Use a pivot table to average the results for each temperature
2. Scatter plot the average of the `peak_strength` measurements as a function of temperature
   - As in Problem 3.3, plot the points with a "peelable" failure mode as an 'x' and points with a "destruct" failure mode as a square
   - Plot a horizontal line at the force where the failure mode changes from peelable to destruct
3. Add a solid line, showing the heat seal curve smoothed by the rolling mean with n=3
4. Additionally, for each temperature calculate the minimum and maximum force that was measured
   - Smooth the min/max values using a rolling mean with n=3
   - Plot this range as a semi-transparent band using the function [`Axes.fill_between(x, y_min, y_max)`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.fill_between.html)
5. Save the plot to the output director as `problem4-3.png`

In [ ]:
# problem 4.3. solution

# load the data
seal_df = pd.read_csv('../../data/seal_curve.csv')
seal_df.head()

In [ ]:
# part 1
# calculate the average value at each temperature
# include the failure mode in the data set so that we can use it for plotting
seal_pt = seal_df.pivot_table(index='temperature', values=['peak_strength', 'failure_mode'], aggfunc='mean')


# part 2
# separate the data into peelable and destruct data sets
# because the failure mode is averaged, allow for non-integer values
peelable_df = seal_pt[seal_pt['failure_mode']<0.5]
destruct_df = seal_pt[seal_pt['failure_mode']>=0.5]

# compute the force and temperature, as done in Problem 3.3
destruct_force = (peelable_df['peak_strength'].max() + destruct_df['peak_strength'].min())/2
initiation_temp = (peelable_df.index.max() + destruct_df.index.min())/2

# smooth the data using a rolling mean, centered in the window
seal_smoothed = seal_pt['peak_strength'].rolling(3, center=True).mean()

fig, ax = plt.subplots()

ax.scatter(peelable_df.index, peelable_df['peak_strength'], marker='x')
ax.scatter(destruct_df.index, destruct_df['peak_strength'], marker='s')

ax.plot(seal_pt.index, seal_smoothed)

ax.set_xlabel('Temperature (°C)')
ax.set_ylabel('Force (N)')


# part 3
ax.axhline(destruct_force, ls='--', zorder=0, alpha=0.5)
ax.text(80, 60, f'Initiation Temperature: {initiation_temp:0.0f}°C')


# part 4: calculate the minimum and maximum values
seal_pt_minmax = seal_df.pivot_table(index='temperature', values='peak_strength', aggfunc=['min', 'max'])

# create a series with the rolling mean smoothed minimum
seal_min = seal_pt_minmax['min']['peak_strength'].rolling(3, center=True).mean()

# create a series with the rolling mean smoothed maximum
seal_max = seal_pt_minmax['max']['peak_strength'].rolling(3, center=True).mean()

# use the fill-between to plot this band
ax.fill_between(seal_pt_minmax.index, seal_min, seal_max, alpha=0.5, zorder=1)


# part 5: save the figure
plt.savefig('../../output/problem4-3-solution.png', dpi=300)

--------------
## Next Steps:

1. Advance to [Unit 5](../05-statistics/unit05-lesson.ipynb) when you're ready for the next step